# Service

1. 必要ファイル及びディレクトリ構造
    1. サービスファイル(.srv)(1つの入出力の組には1つのサービスしか記述できないので似たものであっても違うサービスファイルに分けて定義する)
    ros/catkin_ws/src/_package_name_/srv/_srv_name.srv_
    2. サーバー(サービスノード)  
    ros/catkin_ws/src/_package_name_/src/_file_name.any(ex.py)_
    3. クライアント(クライアントノード:勿論単一のサービスのクライアントを1つのノードとしてもよいし、様々な処理を行うノードの一部にこのクライアントを含めてもよい)  
    ros/catkin_ws/src/_package_name_/srv/_file_name.any(ex.py)_

---

2. ファイル定義
    1. サービスファイル(.srv)の定義
    2. サービスノードの定義
    3. クライアントノードの定義

In [4]:
#サービスファイルの定義
#ファイルは以下のように記述

#データ型 変数名 #入力　
#--- #必用
#データ型 変数名 #出力

#ex)

int64 a
int64 b
---
string response

#もし入力or出力がなかったら何も書かずに---＋どっちか書いてGG

SyntaxError: invalid syntax (<ipython-input-4-0b341a9c0981>, line 10)

In [3]:
#サービスノードの定義

#!/usr/bin/env python   ＃いつもの　PythonROSノードはこれが必要　これでPythonスクリプトであることを認識
from beginner_tutorials.srv import *   #beginner_tutorialsというパッケージ内のsrvディレクトリ内を全参照せよという意味
import rospy

def handle_add_two_ints(req):
    print "Returning [%s + %s = %s]"%(req.a, req.b, (req.a + req.b))
    return AddTwoIntsResponse(req.a + req.b)

def add_two_ints_server():
    rospy.init_node('add_two_ints_server')   #いつもの　ノード名を宣言　これはサービス名ではないことに注意されたい
    
    s = rospy.Service('add_two_ints', AddTwoInts, handle_add_two_ints)
    #AddTwoInts(サービス型)のadd_two_ints(サービス名)というサービスを宣言、更に全てのリクエストをhandle_add_two_ints関数に投げることを宣言
    #ここで注意したいのはhandle_add_two_ints関数はAddTwoIntsRequest(自動生成)のインスタンスに呼び出され、結果はAddTwoIntsResponse(〃)のインスタンスを返す
    #srvファイルを用意した時点で、サービス名＋Request(/Response)は生成されている
    
    print "Ready to add two ints."
    rospy.spin()   #いつもの　サービス完了までコードが終了しないようにしている

if __name__ == "__main__":
    add_two_ints_server()

SyntaxError: invalid syntax (<ipython-input-3-db65b0ef8eae>, line 9)

注意事項:  
- handleのreturnは必ずサービスファイルに定義されたものを返すようにしないとエラーが出る
- あくまで帰ってくるのはAddTwoIntsResponse型のインスタンスなので、ターミナル上ではresponse: ○○みたいに表示される

In [ ]:
#クライアントノードの定義

#!/usr/bin/env python
import sys
import rospy
from beginner_tutorials.srv import *

def add_two_ints_client(x, y):
    rospy.wait_for_service('add_two_ints')   #'add_two_ints'という名前のサービスが立ち上がるまで待機、サーバー側を実行しておかないと一生進まない
    try:   #例外処理
        add_two_ints = rospy.ServiceProxy('add_two_ints', AddTwoInts)   #なんとこれでサーバー側で定義したhandle関数を作製できる
        resp1 = add_two_ints(x, y)   #サーバーののhandleを通常関数のように呼び出せる
        return resp1.sum
    except rospy.ServiceException, e:   #呼出し失敗するとこれが返ってくる
        print "Service call failed: %s"%e

def usage():
    return "%s [x y]"%sys.argv[0]

if __name__ == "__main__":
    if len(sys.argv) == 3:
        x = int(sys.argv[1])
        y = int(sys.argv[2])
    else:
        print usage()
        sys.exit(1)
    print "Requesting %s+%s"%(x, y)
    print "%s + %s = %s"%(x, y, add_two_ints_client(x, y))
    
    
#クライアントはノード宣言rospy.init_node()をする必要はない

注意事項:  
- クライアントはノード宣言rospy.init_node()をする必要はない
- あくまで渡す変数はAddTwoIntsRequestのインスタンス変数なので、表示させるとrequest:○○みたいに表示される
- サービスを呼び出す際に渡す変数は、必ずサービスファイルに定義されたものを使用しないとエラーが出る
- 呼出し失敗時のエラー型はrospy.ServiceExceptionなのでこれを受け取れる例外処理を用意しておく

---

2. 実行前作業
    - サービスの依存関係を明記
        - srvが所属しているパッケージのCMakeLists.txtを以下のように編集 
    - サーバーとクライアントのファイルに実行権限を付加 chmod +x _faile_name_
    - catkin_make

In [ ]:
#コメントアウトを外して、ファイル名を自分で作成したサービスファイルの名前に変更
#add_service_files(
#    FILES
#    <ファイル名>.srv
#    <ファイル名>.srv
#)
↓
add_service_files(
    FILES
    <ファイル名>.srv
    <ファイル名>.srv
)

---

3. 実行

- Command
    - rosrun _package_name_ _node_file_
    - rosservice call _service_name_ _arg1_ ...
    
どちらでも

---

## 参考文献

チュートリアル:
https://wiki.ros.org/ja/ROS/Tutorials/CreatingMsgAndSrv#Creating_a_srv